# TIF345 Advanced Simulation and Machine Learning
## Lab2
Andreas Ekström, 2020

This computer lab is meant to familiarize you with the concept of probabilistic programming and how to use it for doing some Bayesian inference and model selection. We will use the Python module 'pymc3'. It uses Theano for computing gradients via automatic differentiation. Although it compiles probabilistic programs to C for increased speed, it allows model specification directly in Python code. 

From Wikipedia, we read (https://en.wikipedia.org/wiki/Probabilistic_programming)
>Probabilistic programming (PP) is a programming paradigm in which probabilistic models are specified and inference for these models is performed automatically. It represents an attempt to unify probabilistic modeling and traditional general purpose programming in order to make the former easier and more widely applicable. It can be used to create systems that help make decisions in the face of uncertainty. Programming languages used for probabilistic programming are referred to as "probabilistic programming languages" (PPLs).

One should always take claims of "...performed automatically..." and "...widely applicable..." with a grain of salt. Nevertheless, the PP tools offered via e.g. pymc3, or Stan (https://mc-stan.org) which is another very popular PPL, can be very useful for doing Bayesian inference with highly non-trivial, and user-defined, probabilistic models. 

There are additional pymc3 examples in the second notebook 'many_models.ipynb, and the pymc3 website contains useful tutorials (https://docs.pymc.io/nb_tutorials/index.html) and further syntax documentation (https://docs.pymc.io)

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from scipy.stats import uniform, norm

sns.set_context("paper", font_scale=1.5)
sns.set_style("darkgrid")
sns.set_palette("deep")
sns.set(font='sans-serif')
%matplotlib inline
plt.rcParams['figure.dpi'] = 140
import pymc3 as pm
import arviz as az
import corner
import theano.tensor as tt

print(f'pymc3 version: {pm.__version__}')
np.random.seed(123)


# A first example: linear regression

You should use pymc3 to model some outcomes $Y$ as a normally distributed variable with a mean $\mu$ and variance $\sigma^2$. The mean is a _linear_ linear function of two parameters $\theta_0$ and $\theta_1$ (where we sometimes refer to $\theta_0$ as the bias or constant) and a control parameter $x$. The control parameter might be a time, temperature, location etc, for which we have data $Y$. 

We write our model:

\begin{equation}
Y \sim \mathcal{N}(\mu,\sigma^2) \\
\mu = \theta_0 + \theta_1 x
\end{equation}

We will model the observational error with $\sigma$, and employ rather weak priors

\begin{equation}
\theta_i \sim \mathcal{N}(0,10^2) \\
\sigma^2 \sim \mathcal{IG}(1,1).
\end{equation}

## Generate the data

We can use scipy.stats to generate some fake data. We will set the true values $[\theta_0,\theta_1] = [1,2]$ ,consider $x \in [-1,1]$, and add (homoskedastic) zero-mean noise distributed as $\mathcal{N}(0,1)$

In [ ]:
#number of data points
Nd = 50
#true parameter values
thetaT = np.array([1,2])

x = np.linspace(-1,1,Nd)
x_array = np.linspace(-1,1,200)
sigma = 1.0

#here we use the scipy stats norm to generate the noise
Y = thetaT[0] + thetaT[1]*x + norm(0,sigma).rvs(Nd)

## Plot the data

In [ ]:
sns.scatterplot(x,Y,label='Data');
sns.lineplot(x_array,thetaT[0] + thetaT[1]*x_array,color='red',label='Reality',lw=2)
plt.xlabel('x');
plt.ylabel('Y');

## Specify the statistical model (priors and the likelihood) in pymc3

In [ ]:
# This PPL object is a container for all _random_ variables that belong to the model
our_regression_model = pm.Model()

# everything within this Python 'context' will be added to our_regression_model
# and can be handled by pymc3. You cannot create a random variable outside of a model
#
# For example: try creating a random variable by uncommenting the following line, you should get an error
# a_random_variable = pm.Uniform('same_name_as_the_python_variable',0,1)
#
with our_regression_model:

    # Priors for unknown model parameters:
    # an array of two normally distributed model parameters
    theta = pm.Normal('theta', mu=0, sd=10, shape=2)
#     theta = pm.Uniform('theta', lower=2, upper=3, shape=2)
    # the inverse gamma prior for the variance
    sigma2 = pm.InverseGamma('sigma2',1,1)

    # Expected value of model outcome:
    # in our case, the linear model we use to explain the data
    # This part of the calculation is deterministic in the sense that for
    # given values of theta[:], the resulting mu will be computed
    mu = theta[0] + theta[1]*x

    # Likelihood of observations: this is a stochastic variable.
    # We use a standard normal likelihood. Non-standard likelihoods
    # must be explicitly encoded. It is important to remember all normalization constants.
    # We did this in the many_models notebook via the DensityDist API. 
    # You can also use the Potential API.
    Y_obs = pm.Normal('Y_obs', mu=mu, sd=tt.sqrt(sigma2), observed=Y)
    
    #We are done specifying the model.

## Find a (local) MAP using the pymc3 built-in autodiff functionality and BFGS algorithm

In [ ]:
#Having specified the prior and likelihood, we can start operating on the posterior.
#pymc3 offers built-in optimizers to find a (local) maximum a posteriori.
#By default, find_MAP uses the Broyden–Fletcher–Goldfarb–Shanno (BFGS) optimization 
#algorithm to find the maximum of the log-posterior but also allows selection of 
#other optimization algorithms from the scipy.optimize module
#
#pymc3 automatically differentiates you model using automatic differentiation
#(a rather circular statement...). Autodiff is a method for automatically generating
#code for the derivatives of a specified function. It is not the same as 
#symbolic differentiation. Instead, via the chain-rule and operator overloading,
#the autodiff algorithm will construct a fully differentiated function. E.g. it will
#replace np.sin(x) with np.cos(x) throughout the backend code.
#
map_estimate = pm.find_MAP(model=our_regression_model)

print(f'According to the Likelihood and Prior defined above, the BFGS optimizer found the following (local) MAP')
th0 = map_estimate['theta'][0]
th1 = map_estimate['theta'][1]
s2  = map_estimate['sigma2']

print(f'[theta_0,theta_1] = [{th0:.3f},{th1:.3f}] and [sigma2] = {s2:.3f}')
print(f'\nThe true parameter values are [theta_0,theta_1] = {thetaT} and [sigma2] = {sigma**2:.3f}')


## Sample the posterior

In [ ]:
# Let us turn to the true advantage of pymc3: the built-in MCMCM sampler.
# Amongst other, you can select NUTS, Metropolis, or SMC
# NUTS: the HMC No U-Turn Sampler. This is a gradient-based sampling method originally constructed 
# in the lattice field-theory community. It uses Hamiltonian dynamics to take long/uncorrelated Metropolis steps.
#
# NUTS is the default method for MCMC sampling
with our_regression_model:
    trace_NUTS = pm.sample(1000)
    
# Once you have tried NUTS, please try 1000 sequential MC steps
# This is the method we used in the lectures to extract marginal likelihoods
# You can comment out the NUTS sampling above. They do not have to be run simultaneously.
with our_regression_model:
    trace_SMC = pm.sample_smc(1000)

### Inspect the traces

In [ ]:
# you can use the Arviz summary function to get an overview of the posterior
# it will provide you with credibility intervals for the parameters (set by hdi_prob)
with our_regression_model:
    print(az.summary(trace_NUTS, kind='stats', round_to=2,hdi_prob=0.9))

In [ ]:
# you can also use one of Arviz many plot functions to inspect the traces. You should see one lineplot per trace.
# 
# At this stage, go back to replace the theta priors with a uniform pdf \theta ~ U(2,3),
# and inspect the traces. Do you understand what is going on?
###  Answer
### The walkers tries to reach higher probability regions, but since the uniform distribution (prior) cuts off just at the 
### beginning of the interesting region.
#
with our_regression_model:
    az.plot_trace(trace_NUTS);

In [ ]:
# or plot only marginalized posteriors with some mean and highest_density_intervals
with our_regression_model:
    az.plot_posterior(trace_NUTS,figsize=(12, 6),hdi_prob=0.9);

In [ ]:
# you can stack the traces and plot them using corner
samples = np.vstack([trace_NUTS['theta'][:,k] for k in range(0,len(trace_NUTS['theta'][0]))])
samples = np.vstack((samples,trace_NUTS['sigma2'][:])).T
fig_corner = corner.corner(samples,labels =['th0','th1','s2'],show_titles=True);

### Inspect the prior and posterior predictives 

The prior predictive pdf represent model predictions across the prior pdf, i.e. before we have looked at the data and updated the model parameter pdfs. This can be useful for getting a feeling for how the priors play out in the physical model. 

In [ ]:
#You can easily draw from the prior/posterior predictive distributions
with our_regression_model:
    posterior_draws = pm.sample_posterior_predictive(trace_NUTS, var_names=["theta", "sigma2"], 
                                                     samples=400, random_seed=123);
    prior_draws = pm.sample_prior_predictive(var_names=["theta", "sigma2"], 
                                                     samples=400, random_seed=123)
# explicitly draw prior samples, one by one, and plot the model prediction
for idx, theta_sample in enumerate(prior_draws['theta']):
    epsilon = np.random.normal(scale=np.sqrt(prior_draws['sigma2'][idx]))
#     epsilon=0
    
    plt.plot(x_array,theta_sample[0] + theta_sample[1]*x_array + epsilon,color='black',alpha=0.05)
    
plt.title('Prior predictive');
plt.xlabel('x');
plt.ylabel('Y');

fig2 = plt.figure()
# explicitly draw posterior samples, one by one, and plot the model prediction
post_mean_pred = np.zeros(len(x_array))
print(post_mean_pred.shape)
for idx, theta_sample in enumerate(posterior_draws['theta']):
    epsilon = np.random.normal(scale=np.sqrt(posterior_draws['sigma2'][idx]))
    mod = theta_sample[0] + theta_sample[1]*x_array + epsilon
    post_mean_pred += mod
    plt.plot(x_array,mod,color='black',alpha=0.05)
post_mean_pred /= len(posterior_draws['theta'])
sns.scatterplot(x,Y,label='Data',zorder=10);
sns.lineplot(x_array,thetaT[0] + thetaT[1]*x_array,color='red',label='Reality',lw=2,zorder=20);
sns.lineplot(x_array,post_mean_pred,color='blue',label='Average posterior prediction',lw=2,zorder=20);
plt.title('Posterior predictive');
plt.xlabel('x');
plt.ylabel('Y');

## Questions
1. what is the role of epsilon in the above posterior predictive plot. 
##### Answer 
Epsilon, in both cases, is a constant error term that we draw from our current estimate for the variance sigma^2. 
It is our way of explaining the measurement error (which is individual in each data point) with a common error term 
constantly shifting our model output. To do it properly, epsilon should be a vector of draws for each x-value I guess.
But doing so would maybe lead to a better representation of the data, but not fit the model.
2. Try removing it, and observe/explain what happens
##### Answer
If we remove it, we are not modelling the data uncertainty in any extent. Remember, we are drawing samples 
from the posterior for theta and sigma! 
Setting epsilon to zero is the same as marginalizing out the effect of sigma (or epsilon)? From MCMC that is the same thing. 
3. Add the posterior mean prediction to the plot

### Do the same thing but with the Sequential MC and likelihood tempering.

In [ ]:
#if you used SMC, your trace will contain reported log-marginal likelihoods for each chain. 
#We used this for model comparison during the lecture earlier this afternoon
print(trace_SMC.report.log_marginal_likelihood)

#You can easily draw from the prior/posterior predictive distributions
with our_regression_model:
    posterior_draws = pm.sample_posterior_predictive(trace_SMC, var_names=["theta", "sigma2"], 
                                                     samples=400, random_seed=123);
    prior_draws = pm.sample_prior_predictive(var_names=["theta", "sigma2"], 
                                                     samples=400, random_seed=123)
# explicitly draw prior samples, one by one, and plot the model prediction
for idx, theta_sample in enumerate(prior_draws['theta']):
    epsilon = np.random.normal(scale=np.sqrt(prior_draws['sigma2'][idx]))
#     epsilon=0
    
    plt.plot(x_array,theta_sample[0] + theta_sample[1]*x_array + epsilon,color='black',alpha=0.05)
    
plt.title('Prior predictive');
plt.xlabel('x');
plt.ylabel('Y');

fig2 = plt.figure()
# explicitly draw posterior samples, one by one, and plot the model prediction
post_mean_pred = np.zeros(len(x_array))
for idx, theta_sample in enumerate(posterior_draws['theta']):
    epsilon = np.random.normal(scale=np.sqrt(posterior_draws['sigma2'][idx]))
    mod = theta_sample[0] + theta_sample[1]*x_array + epsilon
    post_mean_pred += mod
    plt.plot(x_array,mod,color='black',alpha=0.05)
post_mean_pred /= len(posterior_draws['theta'])
sns.scatterplot(x,Y,label='Data',zorder=10);
sns.lineplot(x_array,thetaT[0] + thetaT[1]*x_array,color='red',label='Reality',lw=2,zorder=20);
sns.lineplot(x_array,thetaT[0] + thetaT[1]*x_array,color='blue',label='Average posterior prediction',lw=2,zorder=20);
plt.title('Posterior predictive');
plt.xlabel('x');
plt.ylabel('Y');

We see basically no difference by using SMC instead of NUTS.

# A second example: is there a second sine component?

Using the code below you can generate 50 data points from the two-sine model, 

\begin{equation}
y(t) = A_1 \sin \left[\left( \frac{t}{P_1} + t_1\right)2\pi\right] + A_2 \sin \left[\left( \frac{t}{P_2} + t_2\right)2\pi\right] \varepsilon,
\end{equation}

where $\varepsilon$ is normally distributed noise $\varepsilon \sim \mathcal{N}(0,\sigma^2)$ with known variance $\sigma^2=1$. 

Looking at this data (a couple of cells below), without knowing that there are in fact 2 sine components, it is _a prioi_ reasonable to employ also a one sine model for interpreting the data, i.e.

\begin{equation}
y(t) = A_1 \sin \left[\left( \frac{t}{P_1} + t_1\right)2\pi\right] + \varepsilon.
\end{equation}

Your **task** is to determine, based on the data, whether one of the sine models actually explains the data better than the other model.

## Define models

In [ ]:
def model_sine1(t, A1, P1, t1):
    return A1 * np.sin((t / P1 + t1) * 2 * np.pi)
params_sine1 = ['A1','P1','t1']

def model_sine2(t, A1, P1, t1, A2, P2, t2):
    return A1 * np.sin((t / P1 + t1) * 2 * np.pi) + A2 * np.sin((t / P2 + t2) * 2 * np.pi)
params_sine2 = ['A1','P1','t1','A2','P2','t2']

## Generate some data...

In [ ]:
np.random.seed(123)

n_data = 50

# time of observations
t = np.random.uniform(0, 5, size=n_data)

# Use these parameter settings.
A1_T = 4.2
P1_T = 3.0
t1_T = 0.0
A2_T = 1.2
P2_T = 1.2
t2_T = 0.4

yerr = 1.0
y = np.random.normal(model_sine2(t, A1_T, P1_T, t1_T, A2_T, P2_T, t2_T), yerr)

## Plot the data

In [ ]:
plt.xlabel('t')
plt.ylabel('y')
plt.errorbar(x=t, y=y, yerr=yerr, marker='o', ls=' ', color='black', ecolor='gray', ms=2)
plt.title('Data')
t_range = np.linspace(0, 5, 1000)

## Define priors and likelihoods in pymc3

you can start with uniform priors $A_i \sim \mathcal{U}(0.5,10)$, $P_i \sim \mathcal{U}(0.5,5)$, and $t_i \sim \mathcal{U}(0,0.5)$

### One-sine model:

In [ ]:
M1_model = pm.Model()

with M1_model:
    # Params ['A1','P1','t1']
    A1 = pm.Uniform('A1', lower=0.5, upper=10, shape=1)
    P1 = pm.Uniform('P1', lower=0.5, upper=5, shape=1)
    t1 = pm.Uniform('t1', lower=0, upper=0.5, shape=1)
    

    # Expected value of model outcome:
#     mu = model_sine1(t, A, P, t1)
    mu = A1 * np.sin((t / P1 + t1) * 2 * np.pi)

    # Likelihood of observations
    y_obs_m1 = pm.Normal('y_obs_m1', mu=mu, sd=yerr, observed=y)

### Two-sine model:

In [ ]:
#your code here
M2_model = pm.Model()
with M2_model:
    # Params ['A1','P1','t1','A2','P2','t2']
    A1 = pm.Uniform('A1', lower=0.5, upper=10, shape=1)
    P1 = pm.Uniform('P1', lower=0.5, upper=5, shape=1)
    t1 = pm.Uniform('t1', lower=0, upper=0.5, shape=1)
    A2 = pm.Uniform('A2', lower=0.5, upper=10, shape=1)
    P2 = pm.Uniform('P2', lower=0.5, upper=5, shape=1)
    t2 = pm.Uniform('t2', lower=0, upper=0.5, shape=1)

    # Expected value of model outcome:
#     mu = model_sine2(t, A[0], P[0], ts[0], A[1], P[1], ts[1])
    mu  = A1 * np.sin((t / P1 + t1) * 2 * np.pi) + A2 * np.sin((t / P2 + t2) * 2 * np.pi)

    # Likelihood of observations
    y_obs_m2 = pm.Normal('y_obs_m2', mu=mu, sd=yerr, observed=y)

## Prior predictive to inspect the prior choices

### One-sine model:

In [ ]:
with M1_model:
    prior_draws = pm.sample_prior_predictive(var_names=params_sine1, 
                                             samples=400, random_seed=123)
# explicitly draw prior samples, one by one, and plot the model prediction

for idx, A_sample in enumerate(prior_draws['A1']):
    epsilon = np.random.normal(scale=np.sqrt(yerr))
    plt.plot(t_range,model_sine1(t_range, A_sample, prior_draws['P1'][idx], prior_draws['t1'][idx]) + epsilon,color='black',alpha=0.05) 
    
sns.scatterplot(t,y,label='Data',zorder=10);
sns.lineplot(t_range,model_sine1(t_range, A1_T, P1_T, t1_T) ,color='red',label='Reality',lw=2,zorder=20);
    
plt.title('Prior predictive, model 1');
plt.xlabel('x');
plt.ylabel('Y');

### Two-sine model:

In [ ]:
with M2_model:
    prior_draws = pm.sample_prior_predictive(var_names=params_sine2, 
                                             samples=400, random_seed=123)
# explicitly draw prior samples, one by one, and plot the model prediction

for idx, A_sample in enumerate(prior_draws['A1']):
#     epsilon = np.random.normal(scale=np.sqrt(prior_draws['sigma2'][idx]))
    epsilon = np.random.normal(scale=np.sqrt(yerr))
    plt.plot(t_range,model_sine2(t_range, A_sample, prior_draws['P1'][idx], prior_draws['t1'][idx], prior_draws['A2'][idx], prior_draws['P2'][idx], prior_draws['t2'][idx] + epsilon),color='black',alpha=0.05)
    
plt.title('Prior predictive, model 2');
plt.xlabel('x');
plt.ylabel('Y');

## Sample posteriors and compute $\log(Z_{M_i})$ with SMC

In [ ]:
def smc_sample(pymc3_model, N=4000):
#As with other sampling methods, running a sampler more than one time is useful 
#to compute diagnostics, SMC is no exception. PyMC3 will try to run at least two 
#SMC chains. Not to confuse with the N Markov chains inside each SMC chain).
    with pymc3_model:
        trace = pm.sample_smc(draws=N,chains=2)
    return trace, trace.report.log_marginal_likelihood

In [ ]:
#sample sine model 1
M1_trace, M1_report = smc_sample(M1_model)

In [ ]:
#sample sine model 2
M2_trace, M2_report = smc_sample(M2_model)

## Analyze results

In [ ]:
def check_trace(trace,params,CI_alpha=0.1):
    print(az.summary(trace, round_to=2, kind='stats',hdi_prob=(1-CI_alpha)))
    axs = az.plot_trace(trace,legend=True);   
    samples = np.vstack(np.array([trace[k] for k in params]).T)
    fig_corner = corner.corner(samples,labels = params, show_titles=True);

In [ ]:
# check traces
check_trace(M1_trace, params_sine1)
check_trace(M2_trace, params_sine2)

In [ ]:
#inspect marginal likelihoods
with M1_model:
    az.plot_posterior(M1_trace,figsize=(12, 6),hdi_prob=0.9);
with M2_model:
    az.plot_posterior(M2_trace,figsize=(12, 6),hdi_prob=0.9);

### Plot posterior predictives

In [ ]:
A1_T = 4.2
P1_T = 3.0
t1_T = 0.0
A2_T = 1.2
P2_T = 1.2
t2_T = 0.4

In [ ]:
# Model 1
with M1_model:
    posterior_draws = pm.sample_posterior_predictive(M1_trace, var_names=params_sine1, 
                                                     samples=400, random_seed=123);
# explicitly draw posterior samples, one by one, and plot the model prediction
post_mean_pred = np.zeros(len(t_range))
for idx, A1 in enumerate(posterior_draws['A1']):
    epsilon = np.random.normal(scale=np.sqrt(yerr))
    mod = model_sine1(t_range, A1, posterior_draws['P1'][idx], posterior_draws['t1'][idx]) + epsilon
    post_mean_pred += mod
    plt.plot(t_range,mod,color='black',alpha=0.05)
post_mean_pred /= len(posterior_draws['A1'])

sns.scatterplot(t,y,label='Data',zorder=10);
sns.lineplot(t_range,model_sine2(t_range, A1_T, P1_T, t1_T, A2_T, P2_T, t2_T) ,color='red',label='Reality',lw=2,zorder=20);
sns.lineplot(t_range,post_mean_pred,color='blue',label='Average posterior prediction',lw=2,zorder=20);
plt.title('Posterior predictive - M1');
plt.xlabel('x');
plt.ylabel('Y');

In [ ]:
# Model 2
with M2_model:
    posterior_draws = pm.sample_posterior_predictive(M2_trace, var_names=params_sine2, 
                                                     samples=400, random_seed=123);
# explicitly draw posterior samples, one by one, and plot the model prediction
post_mean_pred = np.zeros(len(t_range))
for idx, A1 in enumerate(posterior_draws['A1']):
    epsilon = np.random.normal(scale=np.sqrt(yerr))
    mod = model_sine2(t_range, A1, posterior_draws['P1'][idx], posterior_draws['t1'][idx], posterior_draws['A2'][idx], posterior_draws['P2'][idx], posterior_draws['t2'][idx]) + epsilon
    post_mean_pred += mod
    plt.plot(t_range,mod,color='black',alpha=0.05)
post_mean_pred /= len(posterior_draws['A1'])

sns.scatterplot(t,y,label='Data',zorder=10);
sns.lineplot(t_range,model_sine2(t_range, A1_T, P1_T, t1_T, A2_T, P2_T, t2_T) ,color='red',label='Reality',lw=2,zorder=20);
sns.lineplot(t_range,post_mean_pred,color='blue',label='Average posterior prediction',lw=2,zorder=20);
plt.title('Posterior predictive - M2');
plt.xlabel('x');
plt.ylabel('Y');

## Conclusions

We see M1 explaining the data fairly well, once the error bands are included. However, M2 is of course a lot better, almost perfectly matching the reality and following the data really nicely. 

The marginal distributions hone in on the true values nicely (which we ofc can't know). Interestingly, in the M2 case we see that the parameters A, P and t are multimodal, which encompass the fact that our model is made up of two sines, and thus the role of the sines are interchangable given the parameter values. However,
this means that A, P and t are very correlated in this case which is seen in the corner plots; i.e. interchaning A1 with A2 means that we've got to switch both P and t as well. If we didn't need to do this, then our corner plots would have modes at e.g (A1, P2) as well. 

# A third example: N-dimensional double-Gaussian

We will now test the capability of SMC to sample $\log(Z)$ of a bi-modal posterior in some parameter vector $\boldsymbol{\theta}$. We set up this problem using a double-Gaussian likelihood, with identical but known covariance $\boldsymbol{\Sigma}$, and a single-Gaussian parameter prior. This implies that we characterize our data with some vector of two means $[\boldsymbol{\mu}_1,\boldsymbol{\mu}_2]$, each one describing the mean of a Gaussian. Bayes' theorem relates all our quantites

\begin{equation}
p(\boldsymbol{\theta}|[\boldsymbol{\mu}_1,\boldsymbol{\mu}_2]) = \frac{p([\boldsymbol{\mu}_1,\boldsymbol{\mu}_2]|\boldsymbol{\theta},\boldsymbol{\Sigma})p(\boldsymbol{\theta}|\boldsymbol{\mu}_0,\boldsymbol{\Sigma}_0)}{p([\boldsymbol{\mu}_1,\boldsymbol{\mu}_2])},
\end{equation}

where the likelihood is given by

\begin{equation}
p([\boldsymbol{\mu}_1,\boldsymbol{\mu}_2]|\boldsymbol{\theta},\boldsymbol{\Sigma},\delta) = \delta\mathcal{N}(\boldsymbol{\mu}|\boldsymbol{\theta},\boldsymbol{\Sigma}) + (1-\delta)\mathcal{N}(\boldsymbol{\mu}|\boldsymbol{\theta},\boldsymbol{\Sigma}) = \\\\
\frac{\delta}{(2\pi)^{N_p/2}|\boldsymbol{\Sigma}|^{1/2}}\exp\left\{ -\frac{1}{2}(\boldsymbol{\mu}_1 - \boldsymbol{\theta})^T \boldsymbol{\Sigma}^{-1}(\boldsymbol{\mu}_1 - \boldsymbol{\theta})\right\} + \frac{1-\delta}{(2\pi)^{N_p/2}|\boldsymbol{\Sigma}|^{1/2}}\exp\left\{ -\frac{1}{2}(\boldsymbol{\mu}_2 - \boldsymbol{\theta})^T \boldsymbol{\Sigma}^{-1}(\boldsymbol{\mu}_2 - \boldsymbol{\theta})\right\} 
\end{equation}

and the prior is given by

\begin{equation}
p(\boldsymbol{\theta}|\boldsymbol{\mu}_0,\boldsymbol{\Sigma}_0) = \mathcal{N}(\boldsymbol{\theta}|\boldsymbol{\mu}_0,\boldsymbol{\Sigma}_0) = \frac{1}{(2\pi)^{N_p/2}|\boldsymbol{\Sigma}_0|^{1/2}}\exp\left\{ -\frac{1}{2}(\boldsymbol{\mu}_0 - \boldsymbol{\theta})^T \boldsymbol{\Sigma}_0^{-1}(\boldsymbol{\mu}_0 - \boldsymbol{\theta})\right\}.
\end{equation}

We choose this setup since a normally distributed prior is conjugate to a normally distributed likelihood, and the posterior is thus also normally distributed. 

One can show that the posterior is normally distributed with a covariance is given by 

\begin{equation}
\tilde{\boldsymbol{\Sigma}} = (\boldsymbol{\Sigma}^{-1} + \boldsymbol{\Sigma}_0^{-1})^{-1}.
\end{equation}

To obtain the marginal likelihood $p([\boldsymbol{\mu}_1,\boldsymbol{\mu}_2])$, we simply look at all known normalization factors. For Bayes' theorem in general, we have, in order, a prior

\begin{equation}
p(\boldsymbol{\theta}) = \frac{1}{Z_{0}} q(\boldsymbol{\theta}),
\end{equation}

a likelihood

\begin{equation}
p(\boldsymbol{\mathcal{D}}|\boldsymbol{\theta}) = \frac{1}{Z_{\ell}} q(\boldsymbol{\mathcal{D}}|\boldsymbol{\theta}),
\end{equation}

and a posterior

\begin{equation}
p(\boldsymbol{\theta}|\boldsymbol{\mathcal{D}}) = \frac{1}{Z_{\boldsymbol{\theta}}} q(\boldsymbol{\mu}|\boldsymbol{\mathcal{D}}), .
\end{equation}

Where the un-normalized pdf:s are denoted $q(\cdot)$. Therefore, we can identify

\begin{equation}
\frac{1}{Z_{\boldsymbol{\theta}}} q(\boldsymbol{\mu}|\boldsymbol{\mathcal{D}}) = \frac{q(\boldsymbol{\mathcal{D}}|\boldsymbol{\theta}) q(\boldsymbol{\theta})}{Z_{\ell}Z_{0}p(\boldsymbol{\mathcal{D}})} \Rightarrow p(\boldsymbol{\mathcal{D}}) = \frac{Z_{\boldsymbol{\theta}}}{Z_{\ell}Z_{0}}
\end{equation}

In our case, with normal distributions throughout, we have

\begin{equation}
Z_0 = (2\pi)^{N_p/2}|\boldsymbol{\Sigma}_0|^{1/2},\,\,\,\, Z_{\ell} = (2\pi)^{N_p/2}|\boldsymbol{\Sigma}|^{1/2},\,\,\,\, Z_{\boldsymbol{\theta}} = (2\pi)^{N_p/2}|\tilde{\boldsymbol{\Sigma}}|^{1/2}.
\end{equation}

The analytical result for the posterior becomes

\begin{equation}
Z \equiv p(\boldsymbol{\mathcal{D}}) = \frac{|\tilde{\boldsymbol{\Sigma}}|^{1/2}}{(2\pi)^{N_p/2}|\boldsymbol{\Sigma}|^{1/2}|\boldsymbol{\Sigma}_0|^{1/2}} \Rightarrow \log[Z] = \frac{1}{2}\log[|\tilde{\boldsymbol{\Sigma}}|] - \frac{N_p}{2}\log[2\pi] - \frac{1}{2}\log[|\boldsymbol{\Sigma}|] - \frac{1}{2}\log[|\boldsymbol{\Sigma}_0|].
\end{equation}

This result remains the same for a likelihood constructed as a linear combination of two normal likelihoods with identical covariance.

## Questions

Use the write-up above, and the code below to answer the following questions:
0. Do you follow the overarching logic behind deriving the analytical expression for the marginal likelihood?
1. Compare the SMC computation of $\log[Z]$ in $N_p=1$ and $N_p=3$ dimensions. 
2. Inspect the traces, do the results agree with your expectations? (two gaussians of equal size...)
3. Try with $N_p=20$. What happens? How many gaussians do you find? How do they compare?
4. The relative distributions are wrong, but the total marginal likelihood is rather good. What happened? Is this a problem?!
4. Go back to $N_p=3$, and launch the HMC-NUTS samples (the default pymc3 method). How many gaussians do you find?

In [ ]:
#we put two gaussian in Np dimensions
Np = 3

#they are symmetrically located around zero
mu1 = np.ones(Np) * (1)
mu2 = -mu1

sig2_prior = 10**2
Sigma_prior = sig2_prior*np.eye(Np)
Sigma_prior_inv = np.linalg.inv(Sigma_prior)
Sigma_prior_det = np.linalg.det(Sigma_prior)
logZ_prior = (Np/2)*np.log(2*np.pi)+(1/2)*np.log(Sigma_prior_det)

sig2 = 0.05**2
Sigma_likelihood = sig2*np.eye(Np)
#below you can try a random covariance
#S = np.random.rand(Np,Np)
#Sigma_likelihood = sig2*(S.T@S)

Sigma_likelihood_inv = np.linalg.inv(Sigma_likelihood)
Sigma_likelihood_det = np.linalg.det(Sigma_likelihood)
logZ_likelihood = (Np/2)*np.log(2*np.pi)+(1/2)*np.log(Sigma_likelihood_det)

Sigma_posterior = np.linalg.inv(Sigma_likelihood_inv + Sigma_prior_inv)
Sigma_posterior_inv = np.linalg.inv(Sigma_posterior)
Sigma_posterior_det = np.linalg.det(Sigma_posterior)

logZ_posterior = (Np/2)*np.log(2*np.pi)+(1/2)*np.log(Sigma_posterior_det)

print(f'log[Z_prior] = {logZ_prior}')
print(f'log[Z_likelihood] = {logZ_likelihood}')
print(f'log[Z_posterior] = {logZ_posterior}')

log_marginal_likelihood = logZ_posterior - logZ_prior - logZ_likelihood
print(f'log[marginal-likelihood] = {log_marginal_likelihood}')

delta = 0.5  #  with delta of the probability mass in mode [1], mode [2] will get (1-delta) of the probability mass

def double_gaussians(theta):
    log_likelihood_G1 = (-0.5*(Np*tt.log(2*np.pi)+tt.log(Sigma_likelihood_det)+
                               (theta-mu1).T.dot(Sigma_likelihood_inv).dot(theta-mu1)))
    log_likelihood_G2 = (-0.5*(Np*tt.log(2*np.pi)+tt.log(Sigma_likelihood_det)+
                               (theta-mu2).T.dot(Sigma_likelihood_inv).dot(theta-mu2)))
    return pm.math.logsumexp([tt.log(delta) + log_likelihood_G1, tt.log(1-delta) + log_likelihood_G2])

In [ ]:
double_gaussian_model = pm.Model()

with double_gaussian_model:
    theta = pm.Normal("theta", mu=np.zeros_like(mu1), sd = np.sqrt(sig2_prior), shape=Np)
    log_likelihood = pm.Potential("log_likelihood", double_gaussians(theta))

In [ ]:
with double_gaussian_model:
#     trace_double_gaussian = pm.sample_smc(2000)
    
    #try sampling also with a more conventional MCMC samples (HMC-NUTS)
    trace_double_gaussian = pm.sample(2000)

In [ ]:
with double_gaussian_model:
    print(az.summary(trace_double_gaussian, kind='diagnostics',round_to=2))

In [ ]:
with double_gaussian_model:
    az.plot_trace(trace_double_gaussian);

In [ ]:
print(trace_double_gaussian.report.log_marginal_likelihood)
print(f'SMC   log[marginal-likelihood] = {np.mean(trace_double_gaussian.report.log_marginal_likelihood):.2f}')
print(f'EXACT log[marginal-likelihood] = {log_marginal_likelihood:.2f}')

In [ ]:
# you can stack the traces and plot them in corner
samples = np.vstack([trace_double_gaussian['theta'][:,k] for k in range(0,len(trace_double_gaussian['theta'][0]))]).T
fig_corner = corner.corner(samples,show_titles=True);

# Extra tasks and more: Bayesian object detection

With pymc3 you can construct a purely Bayesian approach to detecting discrete signals in a noisy 2d data frame, as long as they are well separated signals. This example is hypothetical, but one could imagine a scenario where researchers have obtained data from some space telescope, and the locations of potential stars are unknown.

This example is taken from https://arxiv.org/pdf/0704.3704.pdf, where also multimodal Nested Sampling (MultiNest) is introduced. You can already now test this approach, for clear and well-separated signals, using the pymc3 SMC sampler, however it does not perform as well as MultiNest when the parameter landscape gets more complicated.

If you would like to move forward and play with this idea, I can recommend to download and install a python version of Multinest from Johannes Buchner's github: https://github.com/JohannesBuchner/PyMultiNest

Johannes also offers a code, called UltraNest, which has a better Jupyter Notebook interface: https://johannesbuchner.github.io/UltraNest/index.html

An example of a next-generation algorithm is PolyChord, with a lite-version available via github:
https://github.com/PolyChord/PolyChordLite
(I have not used this myself yet...)

The SMC implementation might run a bit slow on the JupyterHub, and would therefore recommend to run on a dedicated machine. In particular if you would like to explore Nested Sampling with many live points and a high-resolution data set.

In [ ]:
#this code returns an XY-grid with gaussian 2d signal with amplitude A, Radius R, and mean located at X0,Y0
#the data is a 1d vector to simplify future model comparison
def gaussian_shape(A,X0,Y0,R,XY):
    mu=(XY-[X0,Y0])**2/(-2*R**2)
    return np.array(A*np.exp(np.sum(mu,axis=1)))

In [ ]:
xmin = 0 ; xmax = 10
ymin = 0 ; ymax = 10

Nx = 60
Ny = 60

xscale = Nx/(xmax-xmin)
yscale = Ny/(ymax-ymin)

Xline = np.linspace(xmin, xmax, Nx)
Yline = np.linspace(ymin, ymax, Ny)
X, Y = np.meshgrid(Xline, Yline)
XY = np.dstack((X, Y))
XY_arr = XY.reshape(-1,2)

Rmin = 0.3 ; Rmax = 0.4
Amin = 0.5 ; Amax = 2.0

# how many objects do you want the fake data to have?
Nobjects = 3
objects = []
for object in range(0,Nobjects):
    R = Rmin + np.random.rand()*(Rmax-Rmin)
    A = Amin + np.random.rand()*(Amax-Amin)
    X0 = xmin + np.random.rand()*(xmax-xmin)
    Y0 = ymin + np.random.rand()*(ymax-ymin)
    objects.append([X0,Y0,A,R])

#objects = [[2,3,1,1],[4,7,1,1],[8,2,1,1]]
    
signal = np.zeros((Nx*Ny))
print(f'{Nobjects} Object located at [X,Y,A,R]:')
for object in objects:
    print(object)
    X0 = object[0]
    Y0 = object[1]
    A  = object[2]
    R  = object[3]
    signal += gaussian_shape(A,X0,Y0,R,XY_arr)
    
noise_rms = 0.5
noise = np.random.normal(0,noise_rms,(Nx*Ny))

# you can remove the white noise by commenting the last term out
data = signal + noise

# plot the data
ax = sns.heatmap(data.reshape(Nx,Ny),cmap='magma');
ax.set_xticks(np.linspace(0, Nx, 5))
ax.set_xticklabels(np.linspace(xmin, xmax, 5))
ax.set_yticks(np.linspace(0, Ny, 5))
ax.set_yticklabels(np.linspace(ymin, ymax, 5))
ax.invert_yaxis()
ax.set_xlabel('X');
ax.set_ylabel('Y');
ax.set_title('Data');